In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
import xgboost as xgb
import datetime

pd.set_option('display.float_format',lambda x: '%.5f'%x)


In [2]:
print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close() 

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfTrain = dfTrain.set_index(dfTrain['ID'])
dfPred = dfPred.rename(columns=col_name)
dfPred = dfPred.set_index(dfPred['ID'])
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)
dfPredAll = dfPredAll.set_index(dfPredAll['ID'])

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

Load data...
Done.


In [ ]:
#load features
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')
   
        


In [ ]:
#define the input variables, dummies
predictors = raw_predictors+ex_feat
for var in raw_predictors:
    if dfTrain[var].nunique()==1 and dfTrain[var].isnull().sum()==0:
        predictors.remove(var)
    elif dfTrain[var].nunique()<=5 or 'TOOL' in var:
        predictors.remove(var)
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

In [ ]:
kf = KFold(n_splits=5,shuffle=True)
dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
round=0
for train_index, test_index in kf.split(dfTrain):
    round+=1
    train_X = dfTrain.loc[train_index,predictors]
    test_X = dfTrain.loc[test_index,predictors]
    train_Y = dfTrain.loc[train_index,'Y']
    test_Y = dfTrain.loc[test_index,'Y']
    
    dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
    dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
    param = param = {'max_depth':4, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1.5,'subsample':0.8,
                     'colsample_bytree':0.8}
    evallist  = [(dtrain,'train'),(dtest,'eval')]  
    num_round = 5000
    evals_dict = {}
    model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =50)
    performance_df = pd.DataFrame(evals_dict['eval'])
    bst_tree = len(performance_df)-51
    pred_score = model.predict(dpred,ntree_limit =bst_tree)
    if round==1:
        result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
    else:
        result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')

In [ ]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit.csv',header=False,index=False)